In [1]:
import pygame
import random

# Initialize pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 600, 600
ROWS, COLS = 10, 10
CELL_SIZE = WIDTH // COLS
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (200, 0, 0)        # Snake color
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)      # Ladder color
DARK_GREEN = (0, 100, 0)
IMMUNITY_COLOR = (255, 223, 0)  # Yellow for immunity power-up
PIT_COLOR = (128, 0, 128)      # Purple for pits

# Game window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snakes and Ladders")

# Original snakes and ladders
original_ladders = {3: 22, 8: 30, 28: 84, 58: 77}
original_snakes = {99: 10, 95: 56, 52: 29, 79: 41}

# Dynamic snakes and ladders
dynamic_ladders = {}
dynamic_snakes = {}

# Power-Ups and Pits
power_ups = {5, 25, 55, 70, 85}  # Immunity power-up positions
pits = {13, 41, 56, 89, 98}         # Pit positions

# Player positions and status
player_pos = 1
ai_pos = 1
dice_value = 1
turn = "player"
game_over = False
player_immunity = False
ai_immunity = False

# Board numbering
def get_board_number(row, col):
    if row % 2 == 0:
        return row * COLS + col + 1
    else:
        return row * COLS + (COLS - col)

# Draw the game board
def draw_board():
    screen.fill(WHITE)
    font = pygame.font.Font(None, 24)
    for row in range(ROWS):
        for col in range(COLS):
            cell_num = get_board_number(row, col)
            x, y = col * CELL_SIZE, HEIGHT - (row + 1) * CELL_SIZE
            color = WHITE if cell_num % 2 == 0 else (230, 230, 230)
            pygame.draw.rect(screen, color, (x, y, CELL_SIZE, CELL_SIZE))
            pygame.draw.rect(screen, BLACK, (x, y, CELL_SIZE, CELL_SIZE), 2)
            text = font.render(str(cell_num), True, BLACK)
            screen.blit(text, (x + 10, y + 10))
    draw_ladders()
    draw_snakes()

# Get center of a cell
def get_cell_center(pos):
    row = (pos - 1) // 10
    col = (pos - 1) % 10 if row % 2 == 0 else 9 - (pos - 1) % 10
    x = col * CELL_SIZE + CELL_SIZE // 2
    y = HEIGHT - (row + 1) * CELL_SIZE + CELL_SIZE // 2
    return x, y

# Draw ladders and snakes
def draw_ladders():
    for start, end in {**original_ladders, **dynamic_ladders}.items():
        start_x, start_y = get_cell_center(start)
        end_x, end_y = get_cell_center(end)
        pygame.draw.line(screen, GREEN, (start_x, start_y), (end_x, end_y), 5)

def draw_snakes():
    for start, end in {**original_snakes, **dynamic_snakes}.items():
        start_x, start_y = get_cell_center(start)
        end_x, end_y = get_cell_center(end)
        pygame.draw.line(screen, RED, (start_x, start_y), (end_x, end_y), 5)

# Move player with power-ups, pits, and snakes
def move_player(position, roll, player):
    path = []
    for step in range(1, roll + 1):
        new_pos = position + step
        if new_pos > 100:
            break
        path.append(new_pos)

    if not path:
        return position

    highlight_path(path)
    final_pos = path[-1]

    if final_pos in pits:
        print(f"{player} fell into a pit! {('AI' if player == 'Player' else 'Player')} wins!")
        return 'pit'

    if final_pos in power_ups:
        if player == 'Player':
            global player_immunity
            player_immunity = True
            print("Player got an immunity power-up!")
        else:
            global ai_immunity
            ai_immunity = True
            print("AI got an immunity power-up!")

    if final_pos in original_ladders:
        final_pos = original_ladders[final_pos]
    elif final_pos in dynamic_ladders:
        final_pos = dynamic_ladders[final_pos]

    if final_pos in original_snakes and not (player_immunity if player == 'Player' else ai_immunity):
        final_pos = original_snakes[final_pos]
    elif final_pos in dynamic_snakes and not (player_immunity if player == 'Player' else ai_immunity):
        final_pos = dynamic_snakes[final_pos]

    return final_pos

# Highlight player movement
def highlight_path(path):
    for step in path:
        draw_game()
        px, py = get_cell_center(step)
        pygame.draw.circle(screen, RED, (px, py), 15)
        pygame.display.update()
        pygame.time.delay(200)

# Roll dice with animation
def roll_dice():
    global dice_value
    for _ in range(10):
        dice_value = random.randint(1, 6)
        draw_game()
        pygame.time.delay(50)

# Dynamic snakes and ladders
def update_dynamic_board():
    global dynamic_snakes, dynamic_ladders
    dynamic_snakes.clear()
    dynamic_ladders.clear()

    if player_pos > ai_pos + 10:
        for _ in range(2):
            start = random.randint(max(player_pos - 10, 2), min(player_pos + 10, 99))
            end = random.randint(start - 10, start - 5)
            if start > end and start not in original_snakes:
                dynamic_snakes[start] = end

    if ai_pos > player_pos + 10:
        for _ in range(2):
            start = random.randint(max(ai_pos - 10, 2), min(ai_pos + 10, 99))
            end = random.randint(start - 10, start - 5)
            if start > end and start not in original_snakes:
                dynamic_snakes[start] = end

    if player_pos < ai_pos - 10:
        for _ in range(2):
            start = random.randint(max(player_pos - 5, 2), min(player_pos + 15, 95))
            end = random.randint(start + 5, start + 10)
            if end > start and start not in original_ladders:
                dynamic_ladders[start] = end

    if ai_pos < player_pos - 10:
        for _ in range(2):
            start = random.randint(max(ai_pos - 5, 2), min(ai_pos + 15, 95))
            end = random.randint(start + 5, start + 10)
            if end > start and start not in original_ladders:
                dynamic_ladders[start] = end

# Draw game elements
def draw_power_ups_and_pits():
    for pos in power_ups:
        x, y = get_cell_center(pos)
        pygame.draw.circle(screen, IMMUNITY_COLOR, (x, y), 10)

    for pos in pits:
        x, y = get_cell_center(pos)
        pygame.draw.circle(screen, PIT_COLOR, (x, y), 10)

def draw_game():
    draw_board()
    draw_power_ups_and_pits()
    px, py = get_cell_center(player_pos)
    ax, ay = get_cell_center(ai_pos)
    pygame.draw.circle(screen, BLUE, (px, py), 15)
    pygame.draw.circle(screen, DARK_GREEN, (ax, ay), 15)
    font = pygame.font.Font(None, 36)
    dice_text = font.render(f"Dice: {dice_value}", True, BLACK)
    screen.blit(dice_text, (260, 50))
    pygame.display.update()

# Main game loop
running = True
while running:
    draw_game()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE and turn == "player":
            roll_dice()
            result = move_player(player_pos, dice_value, "Player")
            if result == 'pit':
                print("AI Wins!")
                running = False
                break
            player_pos = result
            if player_pos == 100:
                print("Player Wins!")
                running = False
                break
            turn = "ai"

    if turn == "ai":
        pygame.time.delay(1000)
        roll_dice()
        result = move_player(ai_pos, dice_value, "AI")
        if result == 'pit':
            print("Player Wins!")
            running = False
            break
        ai_pos = result
        if ai_pos == 100:
            print("AI Wins!")
            running = False
            break
        turn = "player"

    update_dynamic_board()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
Player got an immunity power-up!
Player fell into a pit! AI wins!
AI Wins!
